In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [ ]:
#pip --version


pip 19.3.1 from /usr/local/lib/python3.6/dist-packages/pip (python 3.6)


In [ ]:
pip install rawpy 

     |████████████████████████████████| 1.6MB 3.5MB/s 


In [ ]:
!pip uninstall scipy

Uninstalling scipy-1.4.1:
  Would remove:
    /usr/local/lib/python3.6/dist-packages/scipy-1.4.1.dist-info/*
    /usr/local/lib/python3.6/dist-packages/scipy/*
Proceed (y/n)? y
  Successfully uninstalled scipy-1.4.1


In [ ]:
pip install scipy==1.2.0

     |████████████████████████████████| 26.6MB 111kB/s 
ERROR: umap-learn 0.4.3 has requirement scipy>=1.3.1, but you'll have scipy 1.2.0 which is incompatible.
ERROR: tensorflow 2.2.0 has requirement scipy==1.4.1; python_version >= "3", but you'll have scipy 1.2.0 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.


In [ ]:
pip install pillow

In [ ]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
from __future__ import division
import os, scipy.io
import tensorflow as tf
import tensorflow.contrib.slim as slim
import numpy as np
import rawpy
import glob

input_dir = '/gdrive/My Drive/Sony11/Sony/test/test_short/'
gt_dir = '/gdrive/My Drive/Sony11/Sony/test/test_long/'
checkpoint_dir = '/gdrive/My Drive/Sony11/Sony/test/model-4000/'
result_dir = '/gdrive/My Drive/Sony11/Sony/test/result_4000/'

# get test IDs
test_fns = glob.glob(gt_dir + '1*.ARW')
test_ids = [int(os.path.basename(test_fn)[0:5]) for test_fn in test_fns]

DEBUG = 0
if DEBUG == 1:
    save_freq = 2
    test_ids = test_ids[0:5]


def lrelu(x):
    return tf.maximum(x * 0.2, x)


def upsample_and_concat(x1, x2, output_channels, in_channels):
    pool_size = 2
    deconv_filter = tf.Variable(tf.truncated_normal([pool_size, pool_size, output_channels, in_channels], stddev=0.02))
    deconv = tf.nn.conv2d_transpose(x1, deconv_filter, tf.shape(x2), strides=[1, pool_size, pool_size, 1])

    deconv_output = tf.concat([deconv, x2], 3)
    deconv_output.set_shape([None, None, None, output_channels * 2])

    return deconv_output


def network(input):
    conv1 = slim.conv2d(input, 32, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv1_1')
    conv1 = slim.conv2d(conv1, 32, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv1_2')
    pool1 = slim.max_pool2d(conv1, [2, 2], padding='SAME')

    conv2 = slim.conv2d(pool1, 64, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv2_1')
    conv2 = slim.conv2d(conv2, 64, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv2_2')
    pool2 = slim.max_pool2d(conv2, [2, 2], padding='SAME')

    conv3 = slim.conv2d(pool2, 128, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv3_1')
    conv3 = slim.conv2d(conv3, 128, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv3_2')
    pool3 = slim.max_pool2d(conv3, [2, 2], padding='SAME')

    conv4 = slim.conv2d(pool3, 256, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv4_1')
    conv4 = slim.conv2d(conv4, 256, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv4_2')
    pool4 = slim.max_pool2d(conv4, [2, 2], padding='SAME')

    conv5 = slim.conv2d(pool4, 512, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv5_1')
    conv5 = slim.conv2d(conv5, 512, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv5_2')

    up6 = upsample_and_concat(conv5, conv4, 256, 512)
    conv6 = slim.conv2d(up6, 256, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv6_1')
    conv6 = slim.conv2d(conv6, 256, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv6_2')

    up7 = upsample_and_concat(conv6, conv3, 128, 256)
    conv7 = slim.conv2d(up7, 128, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv7_1')
    conv7 = slim.conv2d(conv7, 128, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv7_2')

    up8 = upsample_and_concat(conv7, conv2, 64, 128)
    conv8 = slim.conv2d(up8, 64, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv8_1')
    conv8 = slim.conv2d(conv8, 64, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv8_2')

    up9 = upsample_and_concat(conv8, conv1, 32, 64)
    conv9 = slim.conv2d(up9, 32, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv9_1')
    conv9 = slim.conv2d(conv9, 32, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv9_2')

    conv10 = slim.conv2d(conv9, 12, [1, 1], rate=1, activation_fn=None, scope='g_conv10')
    out = tf.depth_to_space(conv10, 2)
    return out


def pack_raw(raw):
    # pack Bayer image to 4 channels
    im = raw.raw_image_visible.astype(np.float32)
    im = np.maximum(im - 512, 0) / (16383 - 512)  # subtract the black level

    im = np.expand_dims(im, axis=2)
    img_shape = im.shape
    H = img_shape[0]
    W = img_shape[1]

    out = np.concatenate((im[0:H:2, 0:W:2, :],
                          im[0:H:2, 1:W:2, :],
                          im[1:H:2, 1:W:2, :],
                          im[1:H:2, 0:W:2, :]), axis=2)
    return out



sess = tf.Session()
in_image = tf.placeholder(tf.float32, [None, None, None, 4])
gt_image = tf.placeholder(tf.float32, [None, None, None, 3])
out_image = network(in_image)

#G_loss = tf.reduce_mean(tf.abs(out_image - gt_image))
#print("G_loss",G_loss)

saver = tf.train.Saver()
sess.run(tf.global_variables_initializer())
ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
if ckpt:
    print('loaded ' + ckpt.model_checkpoint_path)
    saver.restore(sess, ckpt.model_checkpoint_path)

if not os.path.isdir(result_dir + 'output/'):
    os.makedirs(result_dir + 'output/')

if not os.path.isdir(result_dir + 'pre/'):
    os.makedirs(result_dir + 'pre/')

if not os.path.isdir(result_dir + 'reference/'):
    os.makedirs(result_dir + 'reference/')

print("test_ids:: ",test_ids)
all_avg_loss=[]
for test_id in test_ids:
    # test the first image in each sequence
    print("id::",test_id)
    loss_list=[]
    print("loss_list::",loss_list)
    in_files = glob.glob(input_dir + '%05d_00*.ARW' % test_id)     #add uploaded file name
    for k in range(len(in_files)):
        in_path = in_files[k]
        in_fn = os.path.basename(in_path)
        print("input_image: ",in_fn)
        gt_files = glob.glob(gt_dir + '%05d_00*.ARW' % test_id)    #search for reference image of given id
        gt_path = gt_files[0]
        gt_fn = os.path.basename(gt_path)
        in_exposure = float(in_fn[9:-5])
        gt_exposure = float(gt_fn[9:-5])
        ratio = min(gt_exposure / in_exposure, 300)

        raw = rawpy.imread(in_path)
        input_full = np.expand_dims(pack_raw(raw), axis=0) * ratio     #preprocessed input image
        pre_image = input_full

        im = raw.postprocess(use_camera_wb=True, half_size=False, no_auto_bright=True, output_bps=16)
        # scale_full = np.expand_dims(np.float32(im/65535.0),axis = 0)*ratio
        scale_full = np.expand_dims(np.float32(im / 65535.0), axis=0)

        gt_raw = rawpy.imread(gt_path)
        im = gt_raw.postprocess(use_camera_wb=True, half_size=False, no_auto_bright=True, output_bps=16)
        gt_full = np.expand_dims(np.float32(im / 65535.0), axis=0)     #reference image

        input_full = np.minimum(input_full, 1.0)

        output = sess.run(out_image, feed_dict={in_image: input_full})  
        output = np.minimum(np.maximum(output, 0), 1)  #output image
        #g_loss[ind] = G_current
        #print("Loss=%.3f" % np.mean(g_loss[np.where(g_loss)]))
        loss = np.mean(np.absolute(output - gt_full)) 
        #accuracy=(1-loss)*100
        #print("accuracy: ",accuracy,"%")
        print("loss: ",loss)
        loss_list.append(loss)
        #print("loss_list_of_2_images::",loss_list)

        output = output[0, :, :, :]
        gt_full = gt_full[0, :, :, :]
        pre_image = pre_image[0, :, :,:]
        scale_full = scale_full[0, :, :, :]
        scale_full = scale_full * np.mean(gt_full) / np.mean(
            scale_full)  # scale the low-light image to the same mean of the groundtruth


        

        scipy.misc.toimage(output * 255, high=255, low=0, cmin=0, cmax=255).save(
            result_dir + 'output/%5d_00_%d_out.png' % (test_id, ratio))
        scipy.misc.toimage(pre_image * 255, high=255, low=0, cmin=0, cmax=255).save(
            result_dir + 'pre/%5d_00_%d_scale.png' % (test_id, ratio))
        scipy.misc.toimage(gt_full * 255, high=255, low=0, cmin=0, cmax=255).save(
            result_dir + 'reference/%5d_00_%d_gt.png' % (test_id, ratio))
        
    #print("loss_list_of_images::",loss_list)    
    avg=np.mean(loss_list)
    #print("average loss of images of id %d is %f" % (test_id,avg))
    all_avg_loss.append(avg)

print("all_losses",all_avg_loss)
total_loss=np.mean(all_avg_loss)
total_acc=(1-total_loss)*100
print("total_loss_of_model:::::: ",total_loss)
print("total_accuracy_of_model:::::: ",np.round(total_acc,2),"%")

    

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please use `layer.__call__` method instead.
loaded /gdrive/My Drive/Sony11/Sony/test/model-4000/model.ckpt
INFO:tensorflow:Restoring parameters from /gdrive/My Drive/Sony11/Sony/test/model-4000/model.ckpt
test_ids::  [10093, 10074, 10077, 10082, 10087, 10101, 10103, 10105, 10106, 10111, 10116, 10125, 10126, 10139, 10140, 10162, 10163, 10167, 10170, 10172, 10176, 10003, 10006, 10011, 10016, 10022, 10030, 10032, 10034, 10035, 10040, 10045, 10054, 10055, 10068, 10069, 10178, 10185, 10187, 10191, 10192, 10193, 10198, 10199, 10203, 10213, 10217, 10226, 10227, 10228]
id:: 10093
loss_list:: []
input_image:  1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:176: DeprecationWarning: `toimage` is deprecated!
`toimage` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use Pillow's ``Image.fromarray`` directly instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:178: DeprecationWarning: `toimage` is deprecated!
`toimage` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use Pillow's ``Image.fromarray`` directly instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:180: DeprecationWarning: `toimage` is deprecated!
`toimage` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use Pillow's ``Image.fromarray`` directly instead.


id:: 10074
loss_list:: []
input_image:  10074_00_0.1s.ARW
loss:  0.022874402
id:: 10077
loss_list:: []
input_image:  10077_00_0.1s.ARW
loss:  0.040404197
id:: 10082
loss_list:: []
input_image:  10082_00_0.1s.ARW
loss:  0.024941977
id:: 10087
loss_list:: []
input_image:  10087_00_0.1s.ARW
loss:  0.017142244
id:: 10101
loss_list:: []
input_image:  10101_00_0.1s.ARW
loss:  0.020111095
id:: 10103
loss_list:: []
input_image:  10103_00_0.1s.ARW
loss:  0.026873646
id:: 10105
loss_list:: []
input_image:  10105_00_0.1s.ARW
loss:  0.036360655
id:: 10106
loss_list:: []
input_image:  10106_00_0.1s.ARW
loss:  0.040140714
id:: 10111
loss_list:: []
input_image:  10111_00_0.1s.ARW
loss:  0.02313094
id:: 10116
loss_list:: []
input_image:  10116_00_0.1s.ARW
loss:  0.02724219
id:: 10125
loss_list:: []
input_image:  10125_00_0.1s.ARW
loss:  0.014859583
id:: 10126
loss_list:: []
input_image:  10126_00_0.1s.ARW
loss:  0.014159194
id:: 10139
loss_list:: []
input_image:  10139_00_0.1s.ARW
loss:  0.02287652
id